#Downloading the required packages

In [1]:
!pip3 install backtesting

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip3 install pandas_ta

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


#Importing the required libraries and data

In [3]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import matplotlib.pyplot as plt
from backtesting import Strategy, Backtest
from backtesting.lib import TrailingStrategy, crossover
from backtesting.test import SMA

/home/mahandeep/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mahandeep/.local/lib/python3.10/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/home/mahandeep/.local/lib/python3.10/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
data=pd.read_csv('BTCUSDT-hourly-historical-price.csv', index_col='Unnamed: 0', parse_dates=True)
# data.rename(columns={''})

In [6]:
data

,open,high,low,close,volume,close_time,qav,num_trades,taker_base_vol,taker_quote_vol,ignore,a,b
2017-08-17 04:00:00.000,4261.48,4313.62,4261.32,4308.83,47.181009,1502945999999,202366.138393,171,35.160503,150952.477943,7887.635513,NaN,NaN
2017-08-17 05:00:00.000,4308.83,4328.69,4291.37,4315.32,23.234916,1502949599999,100304.823567,102,21.448071,92608.279728,8039.262402,NaN,NaN
2017-08-17 06:00:00.000,4330.29,4345.45,4309.37,4324.35,7.229691,1502953199999,31282.312670,36,4.802861,20795.317224,8041.760498,NaN,NaN
2017-08-17 07:00:00.000,4316.62,4349.99,4287.41,4349.99,4.443249,1502956799999,19241.058300,25,2.602292,11291.347015,8048.127886,NaN,NaN
2017-08-17 08:00:00.000,4333.32,4377.85,4333.32,4360.69,0.972807,1502960399999,4239.503586,28,0.814655,3552.746817,8018.036500,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-31 19:00:00,4503.00,4619.47,4488.33,4619.43,43.862768,1504713600000,199660.763882,441,29.557031,134755.081987,NaN,NaN,NaN
2022-05-31 20:00:00,4503.00,4619.47,4488.33,4619.43,43.862768,1504713600000,199660.763882,441,29.557031,134755.081987,NaN,NaN,NaN
2022-05-31 21:00:00,4503.00,4619.47,4488.33,4619.43,43.862768,1504713600000,199660.763882,441,29.557031,134755.081987,NaN,NaN,NaN
2022-05-31 22:00:00,4503.00,4619.47,4488.33,4619.43,43.862768,1504713600000,199660.763882,441,29.557031,134755.081987,NaN,NaN,NaN


In [7]:
train = data['2018-01-01':'2022-02-01']
train=train[['open', 'high', 'low', 'close', 'volume']]

In [8]:
train.index

Index(['2018-01-01 00:00:00.000', '2018-01-01 01:00:00.000',
       '2018-01-01 02:00:00.000', '2018-01-01 03:00:00.000',
       '2018-01-01 04:00:00.000', '2018-01-01 05:00:00.000',
       '2018-01-01 06:00:00.000', '2018-01-01 07:00:00.000',
       '2018-01-01 08:00:00.000', '2018-01-01 09:00:00.000',
       ...
       '2022-01-31 14:00:00', '2022-01-31 15:00:00', '2022-01-31 16:00:00',
       '2022-01-31 17:00:00', '2022-01-31 18:00:00', '2022-01-31 19:00:00',
       '2022-01-31 20:00:00', '2022-01-31 21:00:00', '2022-01-31 22:00:00',
       '2022-01-31 23:00:00'],
      dtype='object', length=35690)

#Technical Indicators Implementation

In [9]:
class TechnicalIndicators: #This class contains the methods for creating certain technical indicators, which are crucial in analyzing the dataset.
    def __init__(self):
        self.data=train
        self.data['close_1']=self.data['close'].shift(1) #yesterday's close
        self.data['close1']=self.data['close'].shift(-1) #tommorow's close
        self.data['ret_1']=self.data['close']/self.data['close_1']-1 #yesterday's returns
        self.data['ret1']=self.data['close1']/self.data['close']-1 #future returns
        self.data['tret']=self.data['close']/self.data['open']-1 #today's returns
        self.data['oret']=self.data['open']/self.data['close_1']-1 #overnight returns
        self.data['10-day volatility']=self.data['close'].rolling(window=10).std() #10-day volatility
        self.data.interpolate(method='linear', limit_direction='both', inplace=True) #Fill the empty nan values formed via linear interpolation


    def MACD(self, short_time=9, long_time=26, signal_time=9): #MACD indicator
        data=self.data
        data[f'{short_time}_EMA']=data['close'].ewm(span=short_time, adjust=False).mean()
        data[f'{long_time}_EMA']=data['close'].ewm(span=long_time, adjust=False).mean()
        data['MACD_line']=data[f'{short_time}_EMA']-data[f'{long_time}_EMA'] #The MACD_line will have 9-day EMA - 26-day EMA
        data['MACD_Signal_line']=data['MACD_line'].ewm(span=signal_time, adjust=False).mean() #MACD_Signal_line is 9-day EMA of MACD_line
        data['MACD_Trend']=data['MACD_line']-data['MACD_Signal_line'] #MACD_trend will be the difference of the above two


    def BollingerBands(self, window=14, mul_factor=2):
        data=self.data
        data['BB_Mid_line']=data.close.rolling(window=window).mean()
        data['BB_Upper_band']=data['BB_Mid_line']+(mul_factor*data.close.rolling(window=window).std()) #the default window size is 14 days
        data['BB_Lower_band']=data['BB_Mid_line']-(mul_factor*data.close.rolling(window=window).std()) #the default multiplying factor is 2
        data['Bollinger_Width']=(data['BB_Upper_band']-data['BB_Lower_band'])

    def RSI(self, look_back=14):
        df = self.data.copy()

        # Calculate daily price differences
        df['diff'] = df['close'].diff()

        # Separate gains and losses
        df['Pts_gain'] = df['diff'].apply(lambda x: x if x > 0 else 0)
        df['Pts_lost'] = df['diff'].apply(lambda x: -x if x < 0 else 0)

        # Calculate average gains and losses
        df['Avg_gain'] = df['Pts_gain'].rolling(window=look_back).mean()
        df['Avg_loss'] = df['Pts_lost'].rolling(window=look_back).mean()

        # Avoid division by zero
        df['RS'] = df['Avg_gain'] / (df['Avg_loss'].replace(0, 1e-6))

        # Calculate RSI
        self.data['RSI'] = 100 - (100 / (1 + df['RS']))

        return data


    def CMF(self): #Chaikin Money Flow (gives a good estimate by calculating the money flowed in/out of an asset)
        data=self.data
        data['MF_Multiplier']=(2*data['close']-data['low']-data['high'])/(data['high']-data['low'])
        data['MF_Volume']=data['MF_Multiplier']*data['volume']

    def VWAP(self):
      # Assuming you have OHLC data (Open, High, Low, Close) and Volume in your DataFrame
      # You need to calculate Typical Price and Cumulative Volume first

      # Calculate Typical Price
      df=self.data
      df['Typical_Price'] = (df['high'] + df['low'] + df['close']) / 3

      # Calculate Cumulative Volume
      df['Cumulative_Volume'] = df['volume'].cumsum()

      # Calculate Cumulative TP * Volume
      df['Cumulative_TP*Volume'] = df['Typical_Price'] * df['volume'].cumsum()

      # Calculate VWAP
      df['VWAP'] = df['Cumulative_TP*Volume'] / df['Cumulative_Volume']


      #adv20- Average Daily Volume for the past 20 days
      df['adv20']=df['volume'].rolling(window=20).mean()

      # Drop intermediate columns if not needed
      df = df.drop(['Typical_Price', 'Cumulative_Volume', 'Cumulative_TP*Volume'], axis=1)

tc=TechnicalIndicators()
tc.BollingerBands()
tc.MACD()
tc.CMF()
tc.RSI()
tc.VWAP()

In [10]:
train.rename(columns={'close':'Close','open':'Open', 'high':'High', 'low':'Low', 'volume':'Volume'}, inplace=True)
train.info()


<class 'pandas.core.frame.DataFrame'>
Index: 35690 entries, 2018-01-01 00:00:00.000 to 2022-01-31 23:00:00
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Open                  35690 non-null  float64
 1   High                  35690 non-null  float64
 2   Low                   35690 non-null  float64
 3   Close                 35690 non-null  float64
 4   Volume                35690 non-null  float64
 5   close_1               35690 non-null  float64
 6   close1                35690 non-null  float64
 7   ret_1                 35690 non-null  float64
 8   ret1                  35690 non-null  float64
 9   tret                  35690 non-null  float64
 10  oret                  35690 non-null  float64
 11  10-day volatility     35690 non-null  float64
 12  BB_Mid_line           35677 non-null  float64
 13  BB_Upper_band         35677 non-null  float64
 14  BB_Lower_band         35677 non-null  f

#Check for signals using MACD, RSI, EMA, VWAP and Volumes

In [11]:
def check_macd_crossover(df):
    df['macd_crossover'] = 0  # Initialize with 0
    condition = (df['MACD_Trend'] > 0) & (df['MACD_Trend'].shift(1) < 0)
    df.loc[condition, 'macd_crossover'] = 1
    condition_neg = (df['MACD_Trend'] < 0) & (df['MACD_Trend'].shift(1) > 0)
    df.loc[condition_neg, 'macd_crossover'] = -1
    df['previous_macd_negative'] = df['MACD_Trend'].shift(1) < 0# False mean Crossdown

def check_RSI_cond(df):
    df['RSI_cond']=(df['RSI']<70) & (df['RSI']>30) #True means stock is not in overbought or oversold conditions

def volume_cond(df):
    df['Volume_cond']=df['Volume']>df['adv20'] #More volumes than average means significant investor activity. Might signal an important reversal

def BB_cond(df):
    df['bb_cond']=train['Bollinger_Width'].rolling(window=9).mean() < df['Bollinger_Width'].rolling(window=21).mean() #If the bandwidth is less than average, it mean it is a period of low volatility and hence, better trades

def VWAP_cond(df):
    df['vwap_cond']=df['VWAP'] > df['Close'] # VWAP is a good indicator of the mean level of the asset price. Price normally oscillates about this

def EMA_cond(df):
    df['ema_cond']=df['26_EMA'] > df['Close'] # If short window ema > long window ema -> bullish

check_macd_crossover(train)
check_RSI_cond(train)
volume_cond(train)
BB_cond(train)
VWAP_cond(train)
EMA_cond(train)

# Strategy 1

In [12]:
class MyStrat(TrailingStrategy): #Daily mean reversion

    def init(self):
      super().init()
      self.size=1-1e-20

    def next(self):
      super().next()
      self.ret_1=self.data.ret_1
      if self.ret_1[-1]>0: self.sell(size=self.size)
      else: self.buy(size=self.size)


bt=Backtest(train, MyStrat, cash=1000000, commission=0.0015, exclusive_orders=False)

# Strategy 2

In [17]:
class MyStrat(Strategy): #Golden Cross and Golden Death

    def init(self):
      super().init()
      self.size=1-1e-20
      self.ema1=self.data['9_EMA']
      self.ema2=self.data['26_EMA']

    def next(self):
      # super().next()

      self.cross=self.ema1-self.ema2
      if self.cross[-1]<0 and self.data.bb_cond: self.sell(size=self.size)
      else: self.buy(size=self.size)

bt=Backtest(train, MyStrat, cash=1000000, commission=0.0015, exclusive_orders=False)

# Default Strategy

In [14]:
class MyStrat(Strategy): #default
    n1=10
    n2=20
    def init(self):
      # super().init()
      self.size=1 - 1e-20
      close = self.data.Close
      self.sma1 = self.I(SMA, close, self.n1)
      self.sma2 = self.I(SMA, close, self.n2)
    def next(self):
      super().next()
      if crossover(self.sma1, self.sma2): self.buy(size=self.size)
      elif crossover(self.sma2, self.sma1): self.sell(size=self.size)

bt=Backtest(train, MyStrat, cash=1000000, commission=0.0015, exclusive_orders=False)

#Results

In [18]:
result=bt.run()
result

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [16]:
bt.plot()

RuntimeError: First issue `backtest.run()` to obtain results.